<a href="https://colab.research.google.com/github/sankitshane/DeepLearningTF2/blob/master/Executor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import backend as K
import tensorflow as tf
import cv2
import numpy as np
!pip install mnist
import mnist
%matplotlib inline
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
train_Data, train_Labels = mnist.train_images(), mnist.train_labels()
test_Data, test_Labels = mnist.test_images(), mnist.test_labels()

In [0]:
if K.image_data_format() == "channel_first":
  train_Data = train_Data.reshape((train_Data.shape[0], 1, 28, 28))
  test_Data = test_Data.reshape((test_Data.shape[0], 1, 28, 28))
else:
  train_Data = train_Data.reshape((train_Data.shape[0], 28, 28, 1))
  test_Data = test_Data.reshape((test_Data.shape[0], 28, 28, 1))

In [0]:
train_Data = train_Data.astype('float32') / 255.0
test_Data = test_Data.astype('float32') / 255.0

In [0]:
train_Labels = np_utils.to_categorical(train_Labels, 10)
test_Labels = np_utils.to_categorical(test_Labels, 10)

In [0]:
from keras import initializers, layers
from keras.models import Sequential, Model
from keras.layers import (
    Dropout,
    Input,
    AveragePooling2D,
    GlobalAveragePooling2D,
    MaxPool2D
)
from keras.layers.convolutional import (
    Conv2D,
    MaxPooling2D,
    ZeroPadding2D
)
from keras.layers.core import (
    Activation,
    Flatten,
    Dense
)
from keras.layers.merge import (
    concatenate,
    add
)
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.layers.pooling import GlobalMaxPooling2D

In [0]:
class LetNet:
  @staticmethod
  def build(height, width, channel, classes, activation='relu', weightsPath=None):
    model = Sequential()
    inputShape = (height, width, channel)

    if K.image_data_format() == 'channel_first':
      inputShape = (channel, height, width)
    
    model.add(Conv2D(20, 5, padding='same', input_shape=inputShape))
    model.add(Activation(activation))
    model.add(MaxPooling2D(pool_size=(2, 2) ,strides=(2, 2)))

    model.add(Conv2D(50, 5, padding='same'))
    model.add(Activation(activation))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(Flatten())

    model.add(Dense(500))
    model.add(Activation(activation))

    model.add(Dense(classes))
    model.add(Activation('softmax'))

    if weightsPath is not None:
      model.load_weights = weightsPath

    return model

In [0]:
class AlexNet:
  @staticmethod
  def build(height, width, channel, classes, activation='relu', weightsPath=None):
    model = Sequential()
    inputShape = (height, width, channel)

    if K.image_data_format() == "channel_first":
      inputShape = (channel, height, width)
    
    model.add(Conv2D(96, 11, input_shape=inputShape, strides=(4, 4), padding='valid'))
    model.add(Activation(activation))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), padding='valid'))
    model.add(BatchNormalization())

    model.add(Conv2D(256, 11, strides=(1, 1), padding='valid'))
    model.add(Activation(activation))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), padding='valid'))
    model.add(BatchNormalization())

    model.add(Conv2D(384, 3, strides=(1, 1), padding='valid'))
    model.add(Activation(activation))
    model.add(BatchNormalization())

    model.add(Conv2D(384, 3, strides=(1, 1), padding='valid'))
    model.add(Activation(activation))
    model.add(BatchNormalization())

    model.add(Conv2D(256, 11, strides=(1, 1), padding='valid'))
    model.add(Activation(activation))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), padding='valid'))
    model.add(BatchNormalization())

    model.add(Flatten())

    model.add(Dense(4096, input_shape=(244*244*3,)))
    model.add(Activation(activation))
    model.add(Dropout(0.4))
    model.add(BatchNormalization())

    model.add(Dense(4096))
    model.add(Activation(activation))
    model.add(Dropout(0.4))
    model.add(BatchNormalization())

    model.add(Dense(1000))
    model.add(Activation(activation))
    model.add(Dropout(0.4))
    model.add(BatchNormalization())

    model.add(Dense(classes))
    model.add(Activation('softmax'))

    if weightsPath is not None:
      model.load_weights(weightsPath)

    return model

In [0]:
def kerasModelEx(model, data, epochs, batchSize, lr, loss, metric):
  weights = './'+model+"_weights.hdf5"
  trainData, trainLabels, testData, testLabels = data

  load_model = -1
  opt = SGD(lr=lr)

  shape = testData.shape[1:]
  height_, width_, channel_ = shape[0], shape[1], shape[2]
  classes_ = testLabels.shape[1]

  builder = getattr(globals()[model],'build')
  model = builder(height=height_, width=width_, channel=channel_,
                  classes=classes_, 
                  weightsPath=weights if load_model > 0 else None)
  model.compile(loss=loss, optimizer=opt, metrics=[metric])

  if load_model < 0:
    model.fit(trainData, trainLabels, batch_size=batchSize,
              epochs=epochs, verbose=1)
    (loss, accuracy) = model.evaluate(testData, testLabels,
                                      batch_size=batchSize, verbose=1)
    print("Accuracy as {}".format(accuracy*100))
    print("Loss as {}".format(loss*100))
  

In [14]:
model = kerasModelEx("LetNet", (train_Data, train_Labels, test_Data, test_Labels), 20, 128, 0.01, "categorical_crossentropy", "accuracy")

Epoch 1/20
60000/60000 [==============================] - 10s 159us/step - loss: 1.0702 - accuracy: 0.7207
Epoch 2/20
60000/60000 [==============================] - 2s 41us/step - loss: 0.2693 - accuracy: 0.9210
Epoch 3/20
60000/60000 [==============================] - 2s 41us/step - loss: 0.1922 - accuracy: 0.9434
Epoch 4/20
60000/60000 [==============================] - 2s 41us/step - loss: 0.1485 - accuracy: 0.9564
Epoch 5/20
60000/60000 [==============================] - 2s 41us/step - loss: 0.1218 - accuracy: 0.9640
Epoch 6/20
60000/60000 [==============================] - 2s 41us/step - loss: 0.1035 - accuracy: 0.9693
Epoch 7/20
60000/60000 [==============================] - 2s 41us/step - loss: 0.0903 - accuracy: 0.9729
Epoch 8/20
60000/60000 [==============================] - 2s 41us/step - loss: 0.0806 - accuracy: 0.9765
Epoch 9/20
60000/60000 [==============================] - 2s 41us/step - loss: 0.0744 - accuracy: 0.9780
Epoch 10/20
60000/60000 [============================